In [ ]:
! pip install datasets evaluate transformers rouge-score nltk

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install ipywidgets

In [1]:
import transformers

print(transformers.__version__)

4.37.2


In [3]:
print(transformers.__version__)

4.36.2


In [4]:
model_checkpoint = "t5-small"

In [11]:
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("xsum")
metric = load("rouge")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [12]:
raw_datasets["test"][0].keys()

dict_keys(['document', 'summary', 'id'])

In [13]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [14]:
show_random_elements(raw_datasets["train"])

,document,summary,id
0,Test your news knowledge with our 12 days of Christmas news quizzes.\nThis quiz is day five and asks questions about the month of May 2016.,How much do you remember about the news in Wales over the past 12 months?,38366837
1,"Waterman said he was selling the collection, which included 56 Gauge 1 steam models, to fund apprenticeships at the heritage trust he runs.\nTwo of the models sold for £124,000 each, while another went for £86,800.\nHe said the engines had realised what he wanted and he had raised ""not far off"" the amount he hoped.\nA Beyer Goods, which Waterman said was ""the greatest steam railway engine built in miniature"", was one of the models to fetch £120,000.\nThe sale of 32 of the models fetched a total of £627,229, including buyers' premium.\nThe remaining 24 models failed to reach their reserve price and were not sold, but post-sale offers are continuing to come in, the auctioneers said.\nWaterman described the engines, built from scratch for him, as ""the Fabergé eggs of the railway world"".\nHe said he had decided to sell what amounts to around a tenth of his collection in order to raise enough money to secure the future of the Waterman Railway Heritage Trust, which holds his collection of full-size steam engines.\nHe said the pieces he was selling no longer fitted ""into his wider collection"".\n""I never run that stuff any more,"" he said. ""Everything I'm selling is unique. It was all built for me.\n""They are one-offs. They are not toys - they are works of art. That's why they are so pricey.""\nThe sale was carried out by at Dreweatts and Bloomsbury Auctions.\nThe house's steam and model engineering consultant Michael Matthews said: ""The collection is unique.\n""These models were all built from scratch. They were commissioned over a period of 20 years and Mr Waterman got some of the best builders at the time to work for him - few other people would have had the wealth and enthusiasm to do that.""","An auction of part of the model railway collection owned by record producer Pete Waterman has raised more than £600,000.",32331602


In [15]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('t5-small')

In [17]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [18]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [19]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [40]:
# make dataset smaller by half
tokenized_datasets['train'] = tokenized_datasets['train'].shuffle(seed=42).select(range(100))

In [50]:
tokenized_datasets['validation'] = tokenized_datasets['validation'].shuffle(seed=42).select(range(100))

In [49]:
len(tokenized_datasets['validation'])

11332

In [43]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [52]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [53]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [54]:
# The last thing to define for our Seq2SeqTrainer is how to compute the metrics from the predictions. 
# We need to define a function for this, which will just use the metric we loaded earlier, 
# and we have to do a bit of pre-processing to decode the predictions into texts:

import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [55]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [56]:
trainer.train()

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

TrainOutput(global_step=7, training_loss=3.8635292053222656, metrics={'train_runtime': 264.2064, 'train_samples_per_second': 0.378, 'train_steps_per_second': 0.026, 'train_loss': 3.8635292053222656, 'epoch': 1.0})

In [57]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.41k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/herooooooooo/t5-small-finetuned-xsum/commit/0e511a04eb6a30cc650b5c830dd726110593be88', commit_message='End of training', commit_description='', oid='0e511a04eb6a30cc650b5c830dd726110593be88', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
from transformers import AutoModelForSeq2SeqLM
# example
model = AutoModelForSeq2SeqLM.from_pretrained("herooooooooo/t5-small-finetuned-xsum")

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

c:\Users\kevol\anaconda3\envs\pytorch\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kevol\.cache\huggingface\hub\models--herooooooooo--t5-small-finetuned-xsum. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [3]:
# make a prediction on a given text

text = "Waterman said he was selling the collection, which included 56 Gauge 1 steam models, to fund apprenticeships at the heritage trust he runs.\nTwo of the models sold for £124,000 each, while another went for £86,800.\nHe said the engines had realised what he wanted and he had raised not far off the amount he hoped.\nA Beyer Goods, which Waterman said was the greatest steam railway engine built in miniature, was one of the models to fetch £120,000.\nThe sale of 32 of the models fetched a total of £627,229, including buyers' premium.\nThe remaining 24 models failed to reach their reserve price and were not sold, but post-sale offers are continuing to come in, the auctioneers said.\nWaterman described the engines, built from scratch for him, as the Fabergé eggs of the railway world.\nHe said he had decided to sell what amounts to around a tenth of his collection in order to raise enough money to secure the future of the Waterman Railway Heritage Trust, which holds his collection of full-size steam engines.\nHe said the pieces he was selling no longer fitted into his wider collection.\nI never run that stuff any more, he said. Everything I'm selling is unique. It was all built for me.\nThey are one-offs. They are not toys - they are works of art. That's why they are so pricey.\nThe sale was carried out by at Dreweatts and Bloomsbury Auctions.\nThe house's steam and model engineering consultant Michael Matthews said: The collection is unique.\nThese models were all built from scratch. They were commissioned over a period of 20 years and Mr Waterman got some of the best builders at the time to work for him - few other people would have had the wealth and enthusiasm to do that."
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
# outputs = model.generate(inputs, max_length=150, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
# print(tokenizer.decode(outputs[0]))



In [8]:
# save as json
import json


with open('input.json', 'w') as f:
    json.dump({
        "input_data": inputs.tolist(),
        "params": {
            "max_length": 150,
            "min_length": 5,
            "length_penalty": 2.0,
            "num_beams": 4,
            "early_stopping": True
        }
    }, f)

In [14]:
# Example NumPy array of inputs; replace this with your actual inputs
import numpy as np
inputs = np.array([[21603, 10, 2336, 348, 243, 3, 88, 47, 3014, 8, 1232, 6, 84, 1285, 11526, 12520, 397, 209, 7222, 2250, 6, 12, 3069, 28489, 7, 44, 8, 8681, 2019, 3, 88, 3154, 5, 2759, 13, 8, 2250, 1916, 21, 11140, 357, 13161, 284, 6, 298, 430, 877, 21, 3996, 3840, 6, 6192, 5, 216, 243, 8, 7277, 141, 3, 19007, 125, 3, 88, 1114, 11, 3, 88, 141, 3279, 59, 623, 326, 8, 866, 3, 88, 3, 20964, 5, 71, 493, 7975, 1804, 7, 6, 84, 2336, 348, 243, 47, 8, 4016, 7222, 14421, 1948, 1192, 16, 20955, 6, 47, 80, 13, 8, 2250, 12, 27109, 11140, 13922, 5, 37, 1048, 13, 3538, 13, 8, 2250, 3, 89, 25872, 3, 9, 792, 13, 3996, 948, 2555, 6, 357, 3166, 6, 379, 6094, 31, 3331, 5, 37, 4080, 997, 2250, 4567, 12, 1535, 70, 7866, 594, 11, 130, 59, 1916, 6, 68, 442, 18, 7, 9, 109, 704, 33, 6168, 12, 369, 16, 6, 8, 8493, 15, 277, 243, 5, 2336, 348, 3028, 8, 7277, 6, 1192, 45, 8629, 21, 376, 6, 38, 8, 1699, 2235, 154, 5875, 13, 8, 14421, 296, 5, 216, 243, 3, 88, 141, 1500, 12, 1789, 125, 6201, 12, 300, 3, 9, 3, 324, 189, 13, 112, 1232, 16, 455, 12, 3033, 631, 540, 12, 2451, 8, 647, 13, 8, 2336, 348, 18025, 11523, 5313, 6, 84, 4532, 112, 1232, 13, 423, 18, 7991, 7222, 7277, 5, 216, 243, 8, 2161, 3, 88, 47, 3014, 150, 1200, 9695, 139, 112, 6281, 1232, 5, 27, 470, 661, 24, 2005, 136, 72, 6, 3, 88, 243, 5, 7462, 27, 31, 51, 3014, 19, 775, 5, 94, 47, 66, 1192, 21, 140, 5, 328, 33, 80, 18, 1647, 7, 5, 328, 33, 59, 8723, 3, 18, 79, 33, 930, 13, 768, 5, 466, 31, 7, 572, 79, 33, 78, 594, 63, 5, 37, 1048, 47, 4006, 91, 57, 44, 24348, 1544, 17, 7, 11, 17762, 7, 7165, 23040, 7, 5, 37, 629, 31, 7, 7222, 11, 825, 3867, 7792, 2457, 9771, 7, 243, 10, 37, 1232, 19, 775, 5, 506, 2250, 130, 66, 1192, 45, 8629, 5, 328, 130, 3, 17183, 147, 3, 9, 1059, 13, 460, 203, 11, 1363, 2336, 348, 530, 128, 13, 8, 200, 19334, 44, 8, 97, 12, 161, 21, 376, 3, 18, 360, 119, 151, 133, 43, 141, 8, 5987, 11, 14241, 12, 103, 24, 5, 1]])

data = {
    "input_data": {
        "inputs": inputs.tolist()
    },
    "params": {
        "max_length": 150,
        "min_length": 5,
        "length_penalty": 2.0,
        "num_beams": 4,
        "early_stopping": True
    }
}


In [6]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data =  {
  "input_data": {},
  "params": {}
}

body = str.encode(json.dumps(data))

url = 'https://customhf.westeurope.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = '8opaGj8QBSgCYTukYKylGcuGxYeYRzJx'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'herooooooooo-t5-small-finetun-1' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'[]'


In [10]:
req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print("Response:", result)
except urllib.error.HTTPError as error:
    print("The request failed with status code:", error.code)
    print("Headers:", error.info())
    print("Error Response:", error.read().decode("utf8", 'ignore'))


Response: b'[]'


In [15]:
body = str.encode(json.dumps(data))

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print("Response:", result)
except urllib.error.HTTPError as error:
    print("The request failed with status code:", error.code)
    print("Headers:", error.info())
    print("Error Response:", error.read().decode("utf8", 'ignore'))


The request failed with status code: 424
Headers: server: azureml-frontdoor
date: Mon, 19 Feb 2024 10:05:53 GMT
content-type: application/json
content-length: 92
x-ms-run-function-failed: True
x-ms-server-version: azmlinfsrv/1.0.0
x-ms-request-id: 5f13c4bf-48b0-4a3f-b346-98f18145ceeb
x-request-id: 5f13c4bf-48b0-4a3f-b346-98f18145ceeb
ms-azureml-model-error-reason: model_error
ms-azureml-model-error-statuscode: 500
azureml-model-deployment: herooooooooo-t5-small-finetun-1
connection: close


Error Response: {"message": "An unexpected error occurred in scoring script. Check the logs for more info."}


In [16]:
import urllib.request
import json

# Your updated data dictionary
data = {
    "input_data": {
        "inputs": inputs.tolist()  # Assuming 'inputs' is already defined and formatted correctly
    },
    "params": {
        "max_length": 150,
        "min_length": 5,
        "length_penalty": 2.0,
        "num_beams": 4,
        "early_stopping": True
    }
}

# Correctly encode your data dictionary to JSON and then to bytes
body = str.encode(json.dumps(data))

# Your endpoint URL
url = 'https://customhf.westeurope.inference.ml.azure.com/score'
# Headers including the API key and content type
headers = {
    'Content-Type': 'application/json',
    'Authorization': ('Bearer ' + api_key),  # Ensure your API key is defined
    'azureml-model-deployment': 'herooooooooo-t5-small-finetun-1'  # Specific deployment name if necessary
}

# Initialize the request with the URL, encoded body, and headers
req = urllib.request.Request(url, body, headers)

# Try to open the URL and read the response
try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print("Response:", result.decode('utf-8'))  # Decode the bytes response to string for printing
except urllib.error.HTTPError as error:
    # Handle HTTP errors
    print("The request failed with status code:", error.code)
    print("Headers:", error.info())
    print("Error Response:", error.read().decode("utf8", 'ignore'))


The request failed with status code: 424
Headers: server: azureml-frontdoor
date: Mon, 19 Feb 2024 10:06:43 GMT
content-type: application/json
content-length: 92
x-ms-run-function-failed: True
x-ms-server-version: azmlinfsrv/1.0.0
x-ms-request-id: ed4eb667-1baa-4076-8280-e3cbbb035d52
x-request-id: ed4eb667-1baa-4076-8280-e3cbbb035d52
ms-azureml-model-error-reason: model_error
ms-azureml-model-error-statuscode: 500
azureml-model-deployment: herooooooooo-t5-small-finetun-1
connection: close


Error Response: {"message": "An unexpected error occurred in scoring script. Check the logs for more info."}
